In [21]:
import sys; sys.path.append('../../')
import numpy as np
import pandas as pd
from DataPreparation.DataPreparation import read_data
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from mlpath import mlquest as mlq
from ModelAnalysis import cross_validation
from utils import load_hyperparameters, save_model, get_metrics
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
model_name = "StackingEnsemble"
candidate_models = ['RandomForest']
mlq.start_quest("StackingEnsemble", table_dest="../../", log_defs=False)


# <font color="aqua">Stacking Ensemble</font> Model

### Read the data

In [22]:
x_data_d, y_data_d = mlq.l(read_data)(kind='Numerical', standardize=True)
candidate_opt_params = [load_hyperparameters(model_name) for model_name in candidate_models]

### Initiate model

In [23]:
# TODO Edit candidate_opt_params
# TODO Could tune clf_final
clf_rf = RandomForestClassifier(candidate_opt_params[0])
clf_svm = SVC(candidate_opt_params[1])
clf_log = LogisticRegression(candidate_opt_params[2])
clf_final = LogisticRegression(multi_class='multinomial', max_iter=1000)
clf = mlq.l(StackingClassifier)(estimators=[('rf', clf_rf), ('svm', clf_svm), ('log', clf_log)], final_estimator=clf_final)

### Evaluate Model Bias

In [ ]:
clf.fit(x_data_d, y_data_d)
y_pred = clf.predict(x_data_d)
train_metrics = classification_report(y_data_d, y_pred, digits=3)
train_acc, train_wf1 = get_metrics(train_metrics)
print(train_metrics)

### Evaluate Model Generalization

In [ ]:
loo, kfold=cross_validation(clf, x_data_d, y_data_d, k=[10], n_repeats=[10], random_state=1)

loo_acc=loo.get('loo_acc')
loo_wf1=loo.get('loo_wf1')
repeated_10fold_acc, repeated_10fold_wf1 = kfold.get(f'10-Repeated 10-fold')[:2]

#### Save Model

In [ ]:
save_model(model_name, clf)

### Tracking

In [25]:
mlq.log_metrics(train_acc, train_wf1,loo_acc, loo_wf1, repeated_10fold_acc, repeated_10fold_wf1)
mlq.end_quest()
mlq.show_logs("StackingEnsemble", table_dest="../../", last_k=6)